### rackcli cloud - h4x VCV Rack partial metadata resolution via rackcli cloud

There are two ways to generate a full `catalog.json` for your VCV Rack installed plugins: 1) use the `File.Catalog` menu option in my [VCV Rack fork](https://github.com/dirkleas/Rack), or 2) use my [DLwigglz plugin r4xH4x module]() `catalog (partial)` button along with the [rackcli cloud](https://github.com/dirkleas/rackcloud) service via [rackcli](https://github.com/dirkleas/rackcli). Here's the lowdown on 2).

Workflow to get a full `catalog.json` without forking VCV Rack:

1. one time: download and build or install the binary for DLwigglz
1. one time: download and install rackcli
1. add DLwigglz r4xH4x module to your patch, press `catalog (partial)` button
1. patch `catalog.partial.json` via the [rackcloud](https://github.com/dirkleas/rackcloud) or a local full `catalog.json`

Always use the fullly formed, recent `catalog.json` if your hacking project requires width data -- pretty much anything dealing with layout management or patch composition requires it.

Here's an example of what the `catalog.partial.json` (via `catalog (partial)` button on DLwigglz r4xH4x) and augmented patch files might look like. Sample output has been truncated for covenience -- this was generated from my plugins and I've purchased almost every official module and build/installed many others, last count, alost 100 plugins!

In [1]:
from pprint import pprint as pp
from typing import *
from hashlib import sha256
from itertools import islice
from time import gmtime, strftime
import json, boto3

In [2]:
rackdir = '/Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack'
catalog = json.load(open(rackdir + '/catalog.json'))
partial = json.load(open(rackdir + '/catalog.partial.json')) # DLwigglz r4xHrx output
patch = json.load(open(rackdir + '/dox.vcv'))

For reference, here's how one can also interact with AWS [S3](https://aws.amazon.com/s3/) given appropriate rights using the python3 [boto3](https://boto3.readthedocs.io/en/latest/https://aws.amazon.com/s3/) library. Here's an example of accessing the shared corpus (again, assuming access rights -- you'll need to set up your own Amazon AWS account).

In [3]:
obj = boto3.resource('s3').Object('rackcloud', 'corpus.test.json')
x = json.loads(obj.get()['Body'].read().decode('utf-8')) # read/load corpus
# pp(x)
x[strftime("%Y-%m-%d %H:%M:%S", gmtime())] = 'sup?!?!'

# obj.put(Body=str.encode(json.dumps(x))) # write
# pp(json.loads(obj.get()['Body'].read().decode('utf-8'))) # update corpus

Here's the model for the cloud corpus of all know plugin modules.

In [4]:
corpus = {'sha256(rv,ps,pv,ms)': ('rv', 'ps', 'pv', 'ms', 69)}
corpus = {'0668e58ce7a7aa49b5583aac63a2d73b2b45cf1de20efa5dc7351c4216cdd44d': ('rv', 'ps', 'pv', 'ms', 69)}
print(f'corpus model: {corpus}')

corpus model: {'0668e58ce7a7aa49b5583aac63a2d73b2b45cf1de20efa5dc7351c4216cdd44d': ('rv', 'ps', 'pv', 'ms', 69)}


Now we can "share" our fully-formed `catalog.json` with the world via the cloud service (or manually with curl) -- let's see how to generate the appropriate share request. Remember, this assumes a `catalog.json` generated via the fork, complete with width data.

In [6]:
def hash(rv: str, ps: str, pv: str, ms: str) -> str:
    'hash rack version, plugin slug/version, model slug'
    return(sha256(str.encode('{}|{}|{}|{}'.format(rv, ps, pv, ms))).hexdigest())

def catalog_share_corpus(catalog: Dict) -> Dict[str, Tuple]:
    'generate partial corpus from catalog for sharing with cloud, embed in rackcli'
    return({hash(catalog['applicationVersion'], p['slug'], p['version'], m['slug']): 
            (catalog['applicationVersion'], p['slug'], p['version'], m['slug'], m['width']) \
            for p in catalog['plugins'] for m in p['models']})

def patch_share_corpus(patch: Dict) -> Dict[str, Tuple]:
    'corpus request from a "r4xH4x patch" patch, embed in rackcli'
    return({hash(patch['version'], m['plugin'], m['version'], m['model']):
            (patch['version'], m['plugin'], m['version'], m['model'], m['width']) for m in patch['modules']})

corpus = catalog_share_corpus(catalog)
# corpus = patch_share_corpus(patch)
# pp(corpus)

json.dump(corpus, open('/tmp/share.json', 'w'), indent=2, ensure_ascii=False) # serialize for curl
# manual test: curl -X POST -H "Content-Type: application/json" -d @DATAFILE.JSON URL
# curl -X POST -H "Content-Type: application/json" -d @/tmp/share.json https://9k564tm679.execute-api.us-east-1.amazonaws.com/dev/share | jq

{'0000262960951166f8474540e81c3a8993b4d1132f97498bb3e327a70533ce20': ('0.6.1',
                                                                      'PvC',
                                                                      '0.6.0',
                                                                      'TaHaSaHaN',
                                                                      30),
 '0011578f7c3b542292209a8f21fcd3c8926f699302bfea13f523ea3fdd604a3a': ('0.6.1',
                                                                      'SonusModular',
                                                                      '0.6.1',
                                                                      'Piconoise',
                                                                      60),
 '00e1b0f432fb178151e9b7507ea3f328c4315845aebe17564268a06f9ac0ea32': ('0.6.1',
                                                                      'AS',
                                                  

 '552c3d6516f8c87df5d0c3e7958de860abb74757ce699fe7970c938dba91446f': ('0.6.1',
                                                                      'mscHack',
                                                                      '0.6.2',
                                                                      'Mix_24_4_4',
                                                                      855),
 '55c54c284028ae17ea19df768ab9f1c8d4789e00d586ebadf3a531937d60c587': ('0.6.1',
                                                                      'FrozenWasteland',
                                                                      '0.6.6',
                                                                      'LissajousLFO',
                                                                      195),
 '56671a616d290603ab07098f4f1ba31fa77883d0e9d39440c89cf247de981e90': ('0.6.1',
                                                                      'mental',
                                 

                                                                      150),
 'b116374d0b28345741b07007dbbd5a1464d00ed2101e2d2cd755b48d41972d66': ('0.6.1',
                                                                      'mental',
                                                                      '0.6.3',
                                                                      'MentalBinaryDecoder',
                                                                      75),
 'b158ee68ce03e8ac0497a25124008efbb9be2c5a2f986b2239095f6380188a21': ('0.6.1',
                                                                      'dBiz',
                                                                      '0.6.0',
                                                                      'Utility',
                                                                      120),
 'b1c78204b8375a796caa7742ce9cbf529f0a7c2d7314cc11993b953229c35fa4': ('0.6.1',
                                              

                                                                      60),
 'fe9265a636ea0bef1d319e30d8bc00c722d307bbf01625278ece7df6746afe36': ('0.6.1',
                                                                      'CharredDesert',
                                                                      '0.6.0',
                                                                      'Pan',
                                                                      45),
 'ff36e8b502005bb8b1d9d168701f15221cc4bd10b9258876e3bded33555b02a5': ('0.6.1',
                                                                      'Koralfx-Modules',
                                                                      '0.6.9',
                                                                      'Mixovnik',
                                                                      870)}


The **share** service for accepting catalog submissions accepts corpus models and merges them with the current shared corpus in the cloud. You **will be able to** submit this `cloud_share()` result to the cloud via `rackcli cloud --share` in an **upcoming rackcli release**.Here's are the merge mechanics.

In [6]:
def vet_share_corpus(c: List[Dict]) -> Tuple[Dict, Dict]:
    'verify and return valid and invalid corpus request, len(key)==64 and is_corpus_tuple(value)'
    def is_valid(v): return(len(v) == 5 and isinstance(v[0], str) and isinstance(v[1], str) and 
                            isinstance(v[2], str) and isinstance(v[3], str) and isinstance(v[4], int))
    return({k:v for k,v in c.items() if len(k) == 64 and is_valid(v)}, 
           {k:v for k,v in c.items() if not len(k) == 64 or not is_valid(v)})

def cloud_share(request, corpus):
    'merge request into corpus returning updated corpus and errors, add crowd validation, deploy serverless'
    request, errors = vet_share_corpus(request)
    {k:v for k, v in request.items() if k in corpus.keys()} # only add new plugin modules
    return({**corpus, **request}, errors)

c = dict(islice(corpus.items(), 0, 3))
r = {**dict(islice(corpus.items(), 2, 4)), **{'sha256(rv,ps,pv,ms)': ('rv', 3, 'pv', 'ms')}}
pp(c)
pp(r)
pp(cloud_share(r, c))

{'0668e58ce7a7aa49b5583aac63a2d73b2b45cf1de20efa5dc7351c4216cdd44d': ('0.6.1',
                                                                      'Core',
                                                                      '0.6.1',
                                                                      'QuadMIDIToCVInterface',
                                                                      150),
 '30adfcb6a93f91a428fbe18703a910d249ab41279f8907d9911e54b655b1e75d': ('0.6.1',
                                                                      'Core',
                                                                      '0.6.1',
                                                                      'AudioInterface',
                                                                      150),
 'bb47a317a2fec46ea5b7190b6caa72c8f38be0b8cd33a6d5fcd1fdc3ec48bec3': ('0.6.1',
                                                                      'Core',
                                    

Here is the **sync** service wrapper (client and server) for looking up requested plugins in the cloud to "fix" your `catalog.partial.json`. It's possible that the requestor might have plugins installed that aren't known in the cloud corpus -- cloud_sync() will identify those as "unknowns" to be resolved appropriately.

In [7]:
request = ['hash1', 'hash2']
print(f'corpus request model: {request}')

def cloud_sync_partial_request(partial: Dict) -> List[str]:
    'corpus request for your catalog.partial.json, embed in rackcli'
    return([hash(partial['applicationVersion'], p['slug'], p['version'], m['slug']) \
            for p in partial['plugins'] for m in p['models']])

def vet_sync_corpus(request: List[str]) -> bool:
    return(all(isinstance(hash, str) and len(hash) == 64 for hash in request))

request = cloud_sync_partial_request(partial) + ['d00d'*16,] # include corpus unknow
pp(vet_sync_corpus(request))
# pp(request)
json.dump(request, open('/tmp/sync.json', 'w'), indent=2, ensure_ascii=False)

def cloud_sync(request: List[str], corpus: Dict) -> Tuple[List, List]:
    'request projected corpus and uknowns, deploy serverless'
    if not vet_sync_corpus(request): return({}, request)
    return(({k:v for k, v in corpus.items() if k in request}, 
           [r for r in request if not r in corpus.keys()]))

sync = cloud_sync(request, corpus)
print('unknowns:', sync[1])

# json.dump(request, open('/tmp/sync.json', 'w'), indent=2, ensure_ascii=False) # serialize for curl
# test: curl -X POST -H "Content-Type: application/json" -d @DATAFILE.JSON URL
# curl -X POST -H "Content-Type: application/json" -d @/tmp/sync.json https://9k564tm679.execute-api.us-east-1.amazonaws.com/dev/sync | jq

corpus request model: ['hash1', 'hash2']
True
unknowns: ['d00dd00dd00dd00dd00dd00dd00dd00dd00dd00dd00dd00dd00dd00dd00dd00d']


Lastly, let's "fix" our partial catalog with our response widths. "Unknowns" are currently ignored. This an cause issues if one later assumes all plugins have widths. Perhaps a prudent alternative would be to set "unknown"'s widths to an arbitrary values vs. so use cases accessing all plugin modules and assuming a valid width value will "sorta" work. If said arbitrary default was less than the actual width, the module might appear in a patch as trucated or overlaid by another module.

If you're satisfied with the fixer() output, you could save it to `catalog.json` based on this processing of your `catalog.partial.json`.

In [8]:
def fixer(partial, sync):
    'sync the partial with width metadata, igore unknowns'
    for p in partial['plugins']:
        for m in p['models']:
            try: m['width'] = sync[0][hash(partial['applicationVersion'], p['slug'], p['version'], m['slug'])][4]
            except: pass # ignore "unknowns", would an arbitrary default be "better"?
    return(partial)

# pp(partial)
fixed = fixer(partial, sync)
# pp(fixed)
json.dump(fixed, open(rackdir + '/catalog.fixed.json', 'w'), indent=2, ensure_ascii=False)
# pp(fixed[:3]) # diff catalog.json catalog.fixed.json proves match!